In [8]:
import cv2
import numpy as np
import math
import time


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

    def draw(self, frame, wave=False, walk=False):
        x, y = self.position
        s = self.size

        # Head
        cv2.circle(frame, (x, y - s), s // 4, (0, 0, 0), 2)

        # Body
        cv2.line(frame, (x, y - s // 2), (x, y + s // 2), (0, 0, 0), 2)

        # Arms
        if wave:
            # Waving (right arm moves up and down)
            arm_angle = int(math.sin(self.angle) * s // 2)
            cv2.line(frame, (x, y - s // 2),
                     (x + arm_angle, y - s), (0, 0, 0), 2)
        else:
            # Normal arms
            cv2.line(frame, (x, y - s // 2), (x - s // 2, y), (0, 0, 0), 2)
            cv2.line(frame, (x, y - s // 2), (x + s // 2, y), (0, 0, 0), 2)

        # Legs (move if walking)
        if walk:
            offset = int(math.sin(self.walk_offset) * s // 4)
            cv2.line(frame, (x, y + s // 2),
                     (x - s // 4, y + s + offset), (0, 0, 0), 2)
            cv2.line(frame, (x, y + s // 2),
                     (x + s // 4, y + s - offset), (0, 0, 0), 2)
        else:
            cv2.line(frame, (x, y + s // 2), (x - s // 4, y + s), (0, 0, 0), 2)
            cv2.line(frame, (x, y + s // 2), (x + s // 4, y + s), (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2


def main(action='wave'):  # Change to 'walk' or 'wave'
    figure = StickFigure()
    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action == 'wave' or action=="walk_and_wave"),
            walk=(action == 'walk' or action=="walk_and_wave")
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    main(action='walk_and_wave')
    main(action='wave')  # Change to 'walk' or 'wave'
    main(action='walk')  # Change to 'walk' or 'wave'
    


In [9]:
import cv2
import numpy as np
import math


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

    def draw(self, frame, wave=False, walk=False):
        x, y = self.position
        s = self.size

        # Head
        cv2.circle(frame, (x, y - s), s // 4, (0, 0, 0), 2)

        # Body (neck to pelvis)
        neck = (x, y - s // 2)
        pelvis = (x, y + s // 2)
        cv2.line(frame, neck, pelvis, (0, 0, 0), 2)

        # --- Arms (2 segments) ---
        upper_arm_len = s // 2
        lower_arm_len = s // 2

        if wave:
            # Right arm waving
            shoulder_r = neck
            angle_r_shoulder = math.radians(45 + 30 * math.sin(self.angle))
            elbow_r = (
                int(shoulder_r[0] + upper_arm_len * math.cos(angle_r_shoulder)),
                int(shoulder_r[1] - upper_arm_len * math.sin(angle_r_shoulder))
            )
            angle_r_elbow = math.radians(60 * math.sin(self.angle))
            hand_r = (
                int(elbow_r[0] + lower_arm_len * math.cos(angle_r_elbow)),
                int(elbow_r[1] - lower_arm_len * math.sin(angle_r_elbow))
            )
            cv2.line(frame, shoulder_r, elbow_r, (0, 0, 0), 2)
            cv2.line(frame, elbow_r, hand_r, (0, 0, 0), 2)
        else:
            # Both arms down
            for side in [-1, 1]:  # left and right
                shoulder = neck
                elbow = (x + side * upper_arm_len // 2, y)
                hand = (x + side * upper_arm_len, y + s // 2)
                cv2.line(frame, shoulder, elbow, (0, 0, 0), 2)
                cv2.line(frame, elbow, hand, (0, 0, 0), 2)

        # --- Legs (2 segments) ---
        upper_leg_len = s // 2
        lower_leg_len = s // 2

        for side in [-1, 1]:  # left and right legs
            hip = pelvis
            walk_angle = math.radians(30 * math.sin(self.walk_offset + side * math.pi))
            knee = (
                int(hip[0] + side * upper_leg_len * math.sin(walk_angle)),
                int(hip[1] + upper_leg_len * math.cos(walk_angle))
            )
            ankle = (
                int(knee[0] + side * lower_leg_len * math.sin(walk_angle + 0.5)),
                int(knee[1] + lower_leg_len * math.cos(walk_angle + 0.5))
            )
            cv2.line(frame, hip, knee, (0, 0, 0), 2)
            cv2.line(frame, knee, ankle, (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2


def main(action='walk_and_wave'):
    figure = StickFigure()
    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action in ['wave', 'walk_and_wave']),
            walk=(action in ['walk', 'walk_and_wave'])
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main(action='walk_and_wave')
    main(action='wave')  # Change to 'walk' or 'wave'
    main(action='walk')  # Change to 'walk' or 'wave'
    


In [10]:
import cv2
import numpy as np
import math
import sys


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

    def draw(self, frame, wave=False, walk=False):
        x, y = self.position
        s = self.size

        # Head
        cv2.circle(frame, (x, y - s), s // 4, (0, 0, 0), 2)

        # Body (neck to pelvis)
        neck = (x, y - s // 2)
        pelvis = (x, y + s // 2)
        cv2.line(frame, neck, pelvis, (0, 0, 0), 2)

        # --- Arms (2 segments) ---
        upper_arm_len = s // 2
        lower_arm_len = s // 2

        for side in [-1, 1]:  # -1: left, +1: right
            shoulder = neck
            if wave:
                # Dancing arms: both arms wave with different phases
                base_angle = 45 + 30 * math.sin(self.angle + side * math.pi / 2)
                elbow_angle = 60 * math.sin(self.angle + side * math.pi / 2)

                angle_shoulder = math.radians(base_angle * side)
                elbow = (
                    int(shoulder[0] + upper_arm_len * math.cos(angle_shoulder)),
                    int(shoulder[1] - upper_arm_len * math.sin(angle_shoulder)),
                )

                angle_elbow = math.radians(elbow_angle * side)
                hand = (
                    int(elbow[0] + lower_arm_len * math.cos(angle_elbow)),
                    int(elbow[1] - lower_arm_len * math.sin(angle_elbow)),
                )
            else:
                # Arms down
                elbow = (x + side * upper_arm_len // 2, y)
                hand = (x + side * upper_arm_len, y + s // 2)

            cv2.line(frame, shoulder, elbow, (0, 0, 0), 2)
            cv2.line(frame, elbow, hand, (0, 0, 0), 2)

        # --- Legs (2 segments) ---
        upper_leg_len = s // 2
        lower_leg_len = s // 2

        for side in [-1, 1]:
            hip = pelvis
            if walk:
                walk_angle = math.radians(
                    30 * math.sin(self.walk_offset + side * math.pi)
                )
            else:
                walk_angle = 0

            knee = (
                int(hip[0] + side * upper_leg_len * math.sin(walk_angle)),
                int(hip[1] + upper_leg_len * math.cos(walk_angle)),
            )
            ankle = (
                int(knee[0] + side * lower_leg_len * math.sin(walk_angle + 0.5)),
                int(knee[1] + lower_leg_len * math.cos(walk_angle + 0.5)),
            )

            cv2.line(frame, hip, knee, (0, 0, 0), 2)
            cv2.line(frame, knee, ankle, (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2


def main(action="walk_and_wave"):
    figure = StickFigure()

    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action in ["wave", "walk_and_wave"]),
            walk=(action in ["walk", "walk_and_wave"]),
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    # You can manually change this to 'wave', 'walk', or 'walk_and_wave'

    for action_mode in ["walk_and_wave", "wave", "walk"]:
        # action_mode = 'walk_and_wave'  # ← change this for different animations
        main(action=action_mode)

In [11]:
import cv2
import numpy as np
import math
import sys


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

    def draw(self, frame, wave=False, walk=False):
        x, y = self.position
        s = self.size

        # Head
        head_center = (x, y - s)
        head_radius = s // 4
        cv2.circle(frame, head_center, head_radius, (0, 0, 0), 2)

        # --- Hair (simple arc) ---
        hair_thickness = 2
        hair_radius = head_radius + 2
        start_angle = 200
        end_angle = -20
        cv2.ellipse(frame, head_center, (hair_radius, hair_radius), 0, start_angle, end_angle, (0, 0, 0), hair_thickness)

        # --- Eyes ---
        eye_offset_x = head_radius // 2
        eye_offset_y = head_radius // 3
        eye_radius = 3

        # Simulate blinking
        blink = abs(math.sin(self.angle * 2))  # 0 (closed) to 1 (open)
        eye_height = max(1, int(eye_radius * blink))

        for side in [-1, 1]:
            eye_center = (x + side * eye_offset_x, y - s - eye_offset_y)
            cv2.ellipse(frame, eye_center, (eye_radius, eye_height), 0, 0, 360, (0, 0, 0), -1)

        # --- Eyebrows ---
        brow_length = 10
        brow_offset_y = eye_offset_y + 8
        brow_wave = int(3 * math.sin(self.angle + 1))

        for side in [-1, 1]:
            brow_start = (x + side * (eye_offset_x - 5), y - s - brow_offset_y + brow_wave)
            brow_end = (brow_start[0] + side * brow_length, brow_start[1])
            cv2.line(frame, brow_start, brow_end, (0, 0, 0), 1)

        # --- Smile ---
        smile_center = (x, y - s + head_radius // 2)
        smile_radius = 10
        cv2.ellipse(frame, smile_center, (smile_radius, 5), 0, 0, 180, (0, 0, 0), 1)

        # Body (neck to pelvis)
        neck = (x, y - s // 2)
        pelvis = (x, y + s // 2)
        cv2.line(frame, neck, pelvis, (0, 0, 0), 2)

        # --- Arms (2 segments) ---
        upper_arm_len = s // 2
        lower_arm_len = s // 2

        for side in [-1, 1]:  # -1: left, +1: right
            shoulder = neck
            if wave:
                # Dancing arms: both arms wave with different phases
                base_angle = 45 + 30 * math.sin(self.angle + side * math.pi / 2)
                elbow_angle = 60 * math.sin(self.angle + side * math.pi / 2)

                angle_shoulder = math.radians(base_angle * side)
                elbow = (
                    int(shoulder[0] + upper_arm_len * math.cos(angle_shoulder)),
                    int(shoulder[1] - upper_arm_len * math.sin(angle_shoulder)),
                )

                angle_elbow = math.radians(elbow_angle * side)
                hand = (
                    int(elbow[0] + lower_arm_len * math.cos(angle_elbow)),
                    int(elbow[1] - lower_arm_len * math.sin(angle_elbow)),
                )
            else:
                # Arms down
                elbow = (x + side * upper_arm_len // 2, y)
                hand = (x + side * upper_arm_len, y + s // 2)

            cv2.line(frame, shoulder, elbow, (0, 0, 0), 2)
            cv2.line(frame, elbow, hand, (0, 0, 0), 2)

        # --- Legs (2 segments) ---
        upper_leg_len = s // 2
        lower_leg_len = s // 2

        for side in [-1, 1]:
            hip = pelvis
            if walk:
                walk_angle = math.radians(
                    30 * math.sin(self.walk_offset + side * math.pi)
                )
            else:
                walk_angle = 0

            knee = (
                int(hip[0] + side * upper_leg_len * math.sin(walk_angle)),
                int(hip[1] + upper_leg_len * math.cos(walk_angle)),
            )
            ankle = (
                int(knee[0] + side * lower_leg_len * math.sin(walk_angle + 0.5)),
                int(knee[1] + lower_leg_len * math.cos(walk_angle + 0.5)),
            )

            cv2.line(frame, hip, knee, (0, 0, 0), 2)
            cv2.line(frame, knee, ankle, (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2


def main(action="walk_and_wave"):
    figure = StickFigure()

    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action in ["wave", "walk_and_wave"]),
            walk=(action in ["walk", "walk_and_wave"]),
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    # You can manually change this to 'wave', 'walk', or 'walk_and_wave'

    for action_mode in ["walk_and_wave", "wave", "walk"]:
        # action_mode = 'walk_and_wave'  # ← change this for different animations
        main(action=action_mode)

In [12]:
import cv2
import numpy as np
import math
import sys


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

    def draw(self, frame, wave=False, walk=False):
        x, y = self.position
        s = self.size

        # Head
        head_center = (x, y - s)
        head_radius = s // 4
        cv2.circle(frame, head_center, head_radius, (0, 0, 0), 2)

        # --- Hair: few sparse strands like a balding man ---
        for i in range(-2, 3):
            angle = math.radians(90 + i * 15)
            start = (int(head_center[0] + head_radius * math.cos(angle)),
                     int(head_center[1] - head_radius * math.sin(angle)))
            end = (int(start[0] + 5 * math.cos(angle)),
                   int(start[1] - 5 * math.sin(angle)))
            cv2.line(frame, start, end, (0, 0, 0), 1)

        # --- Eyes ---
        eye_offset_x = head_radius // 2
        eye_offset_y = head_radius // 3
        eye_radius = 3

        # Simulate blinking
        blink = abs(math.sin(self.angle * 2))  # 0 (closed) to 1 (open)
        eye_height = max(1, int(eye_radius * blink))

        for side in [-1, 1]:
            eye_center = (x + side * eye_offset_x, y - s - eye_offset_y)
            cv2.ellipse(frame, eye_center, (eye_radius, eye_height), 0, 0, 360, (0, 0, 0), -1)

        # --- Eyebrows ---
        brow_length = 10
        brow_offset_y = eye_offset_y + 8
        brow_wave = int(3 * math.sin(self.angle + 1))

        for side in [-1, 1]:
            brow_start = (x + side * (eye_offset_x - 5), y - s - brow_offset_y + brow_wave)
            brow_end = (brow_start[0] + side * brow_length, brow_start[1])
            cv2.line(frame, brow_start, brow_end, (0, 0, 0), 1)

        # --- Smile ---
        smile_center = (x, y - s + head_radius // 2)
        smile_radius = 10
        cv2.ellipse(frame, smile_center, (smile_radius, 5), 0, 0, 180, (0, 0, 0), 1)

        # --- Full connected beard (from cheek to cheek) ---
        beard_thickness = 2
        beard_radius_x = head_radius + 2
        beard_radius_y = 12
        beard_center = (x, y - s + head_radius // 2 + 2)
        cv2.ellipse(frame, beard_center, (beard_radius_x, beard_radius_y), 0, 0, 180, (0, 0, 0), beard_thickness)

        # Connect beard to cheeks
        cheek_y = y - s + head_radius // 2
        for side in [-1, 1]:
            cheek_x = x + side * (head_radius - 2)
            cv2.line(frame, (cheek_x, cheek_y), (cheek_x, cheek_y + 10), (0, 0, 0), beard_thickness)


        # --- Neck ---
        neck_top = (x, y - s + head_radius)
        neck_bottom = (x, y - s // 2)
        cv2.line(frame, neck_top, neck_bottom, (0, 0, 0), 2)

        # Body (neck to pelvis)
        pelvis = (x, y + s // 2)
        cv2.line(frame, neck_bottom, pelvis, (0, 0, 0), 2)

        # --- Arms (2 segments) ---
        upper_arm_len = s // 2
        lower_arm_len = s // 2

        for side in [-1, 1]:  # -1: left, +1: right
            shoulder = neck_bottom
            if False and wave:
                # Dancing arms: both arms wave with different phases
                base_angle = 45 + 30 * math.sin(self.angle + side * math.pi / 2)
                elbow_angle = 60 * math.sin(self.angle + side * math.pi / 2)

                angle_shoulder = math.radians(base_angle * side)
                elbow = (
                    int(shoulder[0] + upper_arm_len * math.cos(angle_shoulder)),
                    int(shoulder[1] - upper_arm_len * math.sin(angle_shoulder)),
                )

                angle_elbow = math.radians(elbow_angle * side)
                hand = (
                    int(elbow[0] + lower_arm_len * math.cos(angle_elbow)),
                    int(elbow[1] - lower_arm_len * math.sin(angle_elbow)),
                )
            elif True and wave:
                # Arms swaying left-to-right across the body
                sway = int(upper_arm_len * 0.5 * math.sin(self.angle + side))
                elbow = (
                    shoulder[0] + sway,
                    shoulder[1] + upper_arm_len // 2
                )
                hand = (
                    elbow[0] + sway,
                    elbow[1] + lower_arm_len
                )             
            else:
                # Arms down
                elbow = (x + side * upper_arm_len // 2, y)
                hand = (x + side * upper_arm_len, y + s // 2)

            cv2.line(frame, shoulder, elbow, (0, 0, 0), 2)
            cv2.line(frame, elbow, hand, (0, 0, 0), 2)

        # --- Legs (2 segments) ---
        upper_leg_len = s // 2
        lower_leg_len = s // 2

        for side in [-1, 1]:
            hip = pelvis
            if walk:
                walk_angle = math.radians(
                    30 * math.sin(self.walk_offset + side * math.pi)
                )
            else:
                walk_angle = 0

            knee = (
                int(hip[0] + side * upper_leg_len * math.sin(walk_angle)),
                int(hip[1] + upper_leg_len * math.cos(walk_angle)),
            )
            ankle = (
                int(knee[0] + side * lower_leg_len * math.sin(walk_angle + 0.5)),
                int(knee[1] + lower_leg_len * math.cos(walk_angle + 0.5)),
            )

            cv2.line(frame, hip, knee, (0, 0, 0), 2)
            cv2.line(frame, knee, ankle, (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2


def main(action="walk_and_wave"):
    figure = StickFigure()

    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action in ["wave", "walk_and_wave"]),
            walk=(action in ["walk", "walk_and_wave"]),
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    # You can manually change this to 'wave', 'walk', or 'walk_and_wave'

    for action_mode in ["walk_and_wave", "wave", "walk"]:
        # action_mode = 'walk_and_wave'  # ← change this for different animations
        main(action=action_mode)

In [13]:
import cv2
import numpy as np
import math
import sys


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

    def draw(self, frame, wave=False, walk=False):
        
                # --- Banner ---
        banner_top_left = (0, 20)
        banner_bottom_right = (500, 70)
        banner_color = (200, 230, 255)  # Light blue background
        text_color = (0, 0, 0)          # Black text

        cv2.rectangle(frame, banner_top_left, banner_bottom_right, banner_color, -1)  # Filled banner
        cv2.rectangle(frame, banner_top_left, banner_bottom_right, (0, 0, 0), 2)      # Banner border

        banner_text = "Welcome to Stickman Show!"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.9
        thickness = 2

        text_size = cv2.getTextSize(banner_text, font, font_scale, thickness)[0]
        text_x = (banner_bottom_right[0] - text_size[0]) // 2
        text_y = banner_top_left[1] + (banner_bottom_right[1] - banner_top_left[1]) // 2 + text_size[1] // 2

        cv2.putText(frame, banner_text, (text_x, text_y), font, font_scale, text_color, thickness)


        x, y = self.position
        s = self.size

        # Head
        head_center = (x, y - s)
        head_radius = s // 4
        cv2.circle(frame, head_center, head_radius, (0, 0, 0), 2)

        # --- Hair: few sparse strands like a balding man ---
        for i in range(-2, 3):
            angle = math.radians(90 + i * 15)
            start = (int(head_center[0] + head_radius * math.cos(angle)),
                     int(head_center[1] - head_radius * math.sin(angle)))
            end = (int(start[0] + 5 * math.cos(angle)),
                   int(start[1] - 5 * math.sin(angle)))
            cv2.line(frame, start, end, (0, 0, 0), 1)

        # --- Eyes ---
        eye_offset_x = head_radius // 2
        eye_offset_y = head_radius // 3
        eye_radius = 3

        # Simulate blinking
        blink = abs(math.sin(self.angle * 2))  # 0 (closed) to 1 (open)
        eye_height = max(1, int(eye_radius * blink))

        for side in [-1, 1]:
            eye_center = (x + side * eye_offset_x, y - s - eye_offset_y)
            cv2.ellipse(frame, eye_center, (eye_radius, eye_height), 0, 0, 360, (0, 0, 0), -1)

        # --- Eyebrows ---
        brow_length = 10
        brow_offset_y = eye_offset_y + 8
        brow_wave = int(3 * math.sin(self.angle + 1))

        for side in [-1, 1]:
            brow_start = (x + side * (eye_offset_x - 5), y - s - brow_offset_y + brow_wave)
            brow_end = (brow_start[0] + side * brow_length, brow_start[1])
            cv2.line(frame, brow_start, brow_end, (0, 0, 0), 1)

        # --- Smile ---
        smile_center = (x, y - s + head_radius // 2)
        smile_radius = 10
        cv2.ellipse(frame, smile_center, (smile_radius, 5), 0, 0, 180, (0, 0, 0), 1)

        # --- Full connected beard (from cheek to cheek) ---
        beard_thickness = 2
        beard_radius_x = head_radius + 2
        beard_radius_y = 12
        beard_center = (x, y - s + head_radius // 2 + 2)
        cv2.ellipse(frame, beard_center, (beard_radius_x, beard_radius_y), 0, 0, 180, (0, 0, 0), beard_thickness)

        # Connect beard to cheeks
        cheek_y = y - s + head_radius // 2
        for side in [-1, 1]:
            cheek_x = x + side * (head_radius - 2)
            cv2.line(frame, (cheek_x, cheek_y), (cheek_x, cheek_y + 10), (0, 0, 0), beard_thickness)


        # --- Neck ---
        neck_top = (x, y - s + head_radius)
        neck_bottom = (x, y - s // 2)
        cv2.line(frame, neck_top, neck_bottom, (0, 0, 0), 2)

        # Body (neck to pelvis)
        pelvis = (x, y + s // 2)
        cv2.line(frame, neck_bottom, pelvis, (0, 0, 0), 2)

        # --- Arms (2 segments) ---
        upper_arm_len = s // 2
        lower_arm_len = s // 2

        for side in [-1, 1]:  # -1: left, +1: right
            shoulder = neck_bottom
            if False and wave:
                # Dancing arms: both arms wave with different phases
                base_angle = 45 + 30 * math.sin(self.angle + side * math.pi / 2)
                elbow_angle = 60 * math.sin(self.angle + side * math.pi / 2)

                angle_shoulder = math.radians(base_angle * side)
                elbow = (
                    int(shoulder[0] + upper_arm_len * math.cos(angle_shoulder)),
                    int(shoulder[1] - upper_arm_len * math.sin(angle_shoulder)),
                )

                angle_elbow = math.radians(elbow_angle * side)
                hand = (
                    int(elbow[0] + lower_arm_len * math.cos(angle_elbow)),
                    int(elbow[1] - lower_arm_len * math.sin(angle_elbow)),
                )
            elif True and wave:
                # Arms swaying left-to-right across the body
                sway = int(upper_arm_len * 0.5 * math.sin(self.angle + side))
                elbow = (
                    shoulder[0] + sway,
                    shoulder[1] + upper_arm_len // 2
                )
                hand = (
                    elbow[0] + sway,
                    elbow[1] + lower_arm_len
                )             
            else:
                # Arms down
                elbow = (x + side * upper_arm_len // 2, y)
                hand = (x + side * upper_arm_len, y + s // 2)

            cv2.line(frame, shoulder, elbow, (0, 0, 0), 2)
            cv2.line(frame, elbow, hand, (0, 0, 0), 2)

        # --- Legs (2 segments) ---
        upper_leg_len = s // 2
        lower_leg_len = s // 2

        for side in [-1, 1]:
            hip = pelvis
            if walk:
                walk_angle = math.radians(
                    30 * math.sin(self.walk_offset + side * math.pi)
                )
            else:
                walk_angle = 0

            knee = (
                int(hip[0] + side * upper_leg_len * math.sin(walk_angle)),
                int(hip[1] + upper_leg_len * math.cos(walk_angle)),
            )
            ankle = (
                int(knee[0] + side * lower_leg_len * math.sin(walk_angle + 0.5)),
                int(knee[1] + lower_leg_len * math.cos(walk_angle + 0.5)),
            )

            cv2.line(frame, hip, knee, (0, 0, 0), 2)
            cv2.line(frame, knee, ankle, (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2


def main(action="walk_and_wave"):
    figure = StickFigure()

    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action in ["wave", "walk_and_wave"]),
            walk=(action in ["walk", "walk_and_wave"]),
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    # You can manually change this to 'wave', 'walk', or 'walk_and_wave'

    for action_mode in ["walk_and_wave", "wave", "walk"]:
        # action_mode = 'walk_and_wave'  # ← change this for different animations
        main(action=action_mode)

In [14]:
import cv2
import numpy as np
import math
import sys


class StickFigure:
    def __init__(self, position=(250, 300), size=100):
        self.position = position
        self.size = size
        self.angle = 0
        self.walk_offset = 0

        self.logo_howsam = cv2.imread("data/image_howsam.png")
        self.logo_opencv = cv2.imread("data/image_opencv.png")

        self.color_cnt = 0
        self.color_val = (50, 50, 50)

    def draw(self, frame, wave=False, walk=False):

        # --- Banner ---
        banner_top_left = (0, 20)
        banner_bottom_right = (500, 70)
        banner_color = (200, 230, 255)  # Light blue background
        text_color = (0, 0, 0)  # Black text

        cv2.rectangle(
            frame, banner_top_left, banner_bottom_right, banner_color, -1
        )  # Filled banner
        cv2.rectangle(
            frame, banner_top_left, banner_bottom_right, (0, 0, 0), 2
        )  # Banner border

        banner_text = "Welcome to Howsam.org!!!"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.9
        thickness = 2

        text_size = cv2.getTextSize(banner_text, font, font_scale, thickness)[0]
        text_x = (banner_bottom_right[0] - text_size[0]) // 2
        text_y = (
            banner_top_left[1]
            + (banner_bottom_right[1] - banner_top_left[1]) // 2
            + text_size[1] // 2
        )

        cv2.putText(
            frame,
            banner_text,
            (text_x, text_y),
            font,
            font_scale,
            text_color,
            thickness,
        )

        x, y = self.position
        s = self.size

        # Head
        head_center = (x, y - s)
        head_radius = s // 4
        cv2.circle(frame, head_center, head_radius, (0, 255, 255), -1)
        cv2.circle(frame, head_center, head_radius, (0, 0, 0), 2)

        # --- Hair: few sparse strands like a balding man ---
        for i in range(-2, 3):
            angle = math.radians(90 + i * 15)
            start = (
                int(head_center[0] + head_radius * math.cos(angle)),
                int(head_center[1] - head_radius * math.sin(angle)),
            )
            end = (
                int(start[0] + 5 * math.cos(angle)),
                int(start[1] - 5 * math.sin(angle)),
            )
            cv2.line(frame, start, end, (0, 0, 0), 1)

        # --- Eyes ---
        eye_offset_x = head_radius // 2
        eye_offset_y = head_radius // 3
        eye_radius = 3

        # Simulate blinking
        blink = abs(math.sin(self.angle * 2))  # 0 (closed) to 1 (open)
        eye_height = max(1, int(eye_radius * blink))

        for side in [-1, 1]:
            eye_center = (x + side * eye_offset_x, y - s - eye_offset_y)
            cv2.ellipse(
                frame, eye_center, (eye_radius, eye_height), 0, 0, 360, (255, 0, 0), -1
            )

        # --- Eyebrows ---
        brow_length = 10
        brow_offset_y = eye_offset_y + 8
        brow_wave = int(3 * math.sin(self.angle + 1))

        for side in [-1, 1]:
            brow_start = (
                x + side * (eye_offset_x - 5),
                y - s - brow_offset_y + brow_wave,
            )
            brow_end = (brow_start[0] + side * brow_length, brow_start[1])
            cv2.line(frame, brow_start, brow_end, (0, 0, 0), 1)

        # # --- Smile ---
        smile_center = (x, y - s + head_radius // 2)
        smile_radius = 10
        rotation_angle = (
            0 * 45 * math.sin(self.angle)
        )  # oscillate between -45 and +45 degrees
        # cv2.ellipse(frame, smile_center, (smile_radius, 5), 0, 0, 180, (0, 0, 0), 1)

        scale = 10 + 5 * math.sin(self.angle)  # oscillates between 5 and 15
        vertical_scale = 5 + 2 * math.sin(self.angle)

        cv2.ellipse(
            frame,
            smile_center,
            (int(scale), int(vertical_scale)),
            rotation_angle,
            0,
            180,
            (0, 0, 255),
            2,
        )

        # # --- Full connected beard (from cheek to cheek) ---
        # beard_thickness = 2
        # beard_radius_x = head_radius + 2
        # beard_radius_y = 12
        # beard_center = (x, y - s + head_radius // 2 + 2)
        # cv2.ellipse(frame, beard_center, (beard_radius_x, beard_radius_y), 0, 0, 180, (0, 0, 0), beard_thickness)

        # # Connect beard to cheeks
        # cheek_y = y - s + head_radius // 2
        # for side in [-1, 1]:
        #     cheek_x = x + side * (head_radius - 2)
        #     cv2.line(frame, (cheek_x, cheek_y), (cheek_x, cheek_y + 10), (0, 0, 0), beard_thickness)

        # --- Neck ---
        neck_top = (x, y - s + head_radius)
        neck_bottom = (x, y - s // 2)
        cv2.line(frame, neck_top, neck_bottom, (0, 0, 0), 2)

        # Body (neck to pelvis)
        pelvis = (x, y + s // 2)
        cv2.line(frame, neck_bottom, pelvis, (0, 0, 0), 2)

        # --- Arms (2 segments) ---
        upper_arm_len = s // 2
        lower_arm_len = s // 2

        for side in [-1, 1]:  # -1: left, +1: right
            shoulder = neck_bottom
            if False and wave:
                # Dancing arms: both arms wave with different phases
                base_angle = 45 + 30 * math.sin(self.angle + side * math.pi / 2)
                elbow_angle = 60 * math.sin(self.angle + side * math.pi / 2)

                angle_shoulder = math.radians(base_angle * side)
                elbow = (
                    int(shoulder[0] + upper_arm_len * math.cos(angle_shoulder)),
                    int(shoulder[1] - upper_arm_len * math.sin(angle_shoulder)),
                )

                angle_elbow = math.radians(elbow_angle * side)
                hand = (
                    int(elbow[0] + lower_arm_len * math.cos(angle_elbow)),
                    int(elbow[1] - lower_arm_len * math.sin(angle_elbow)),
                )
            elif False and wave:
                # Arms swaying left-to-right across the body
                sway = int(upper_arm_len * 0.5 * math.sin(self.angle + side))
                elbow = (shoulder[0] + sway, shoulder[1] + upper_arm_len // 2)
                hand = (elbow[0] + sway, elbow[1] + lower_arm_len)
            elif True and wave:
                # Arms swaying left-to-right across the body
                sway = int(upper_arm_len * 0.5 * math.sin(self.angle + side))
                elbow = (shoulder[0] + sway, shoulder[1] + upper_arm_len // 2)
                hand = (elbow[0] + sway, elbow[1] + lower_arm_len)
            else:
                # Arms down
                elbow = (x + side * upper_arm_len // 2, y)
                hand = (x + side * upper_arm_len, y + s // 2)

            # Draw arm segments
            cv2.line(frame, shoulder, elbow, (0, 0, 0), 2)
            cv2.line(frame, elbow, hand, (0, 0, 0), 2)

            # --- Hand Banners ---
            banner_width, banner_height = 80, 80
            banner_text = "Right" if side == 1 else "OpenCV"

            banner_top_left = (
                hand[0] - banner_width // 2,
                hand[1] - banner_height - 10,
            )
            banner_bottom_right = (
                banner_top_left[0] + banner_width,
                banner_top_left[1] + banner_height,
            )

            # Draw banner rectangle
            cv2.rectangle(
                frame, banner_top_left, banner_bottom_right, (255, 230, 200), -1
            )  # Fill
            cv2.rectangle(
                frame, banner_top_left, banner_bottom_right, (0, 0, 0), 1
            )  # Border

            # Draw banner text
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            thickness = 2
            text_size = cv2.getTextSize(banner_text, font, font_scale, thickness)[0]

            if side == -1:
                logo = cv2.resize(self.logo_opencv, (banner_width, banner_height))
                frame[
                    banner_top_left[1] : banner_bottom_right[1],
                    banner_top_left[0] : banner_bottom_right[0],
                ] = logo[:, :]
                text_x = banner_top_left[0] + (banner_width - text_size[0]) // 2
                text_y = banner_top_left[1] + (text_size[1]) - 4
                cv2.putText(
                    frame,
                    banner_text,
                    (text_x, text_y),
                    font,
                    font_scale,
                    self.color_val,
                    thickness,
                )
            else:
                logo = cv2.resize(self.logo_howsam, (banner_width, banner_height))
                frame[
                    banner_top_left[1] : banner_bottom_right[1],
                    banner_top_left[0] : banner_bottom_right[0],
                ] = logo[:, :]
                text_x = banner_top_left[0] + (banner_width - text_size[0]) // 2
                text_y = banner_top_left[1] + (banner_height + text_size[1]) // 2 - 4

        # --- Legs (2 segments) ---
        upper_leg_len = s // 2
        lower_leg_len = s // 2

        for side in [-1, 1]:
            hip = pelvis
            if walk:
                walk_angle = math.radians(
                    30 * math.sin(self.walk_offset + side * math.pi)
                )
            else:
                walk_angle = 0

            knee = (
                int(hip[0] + side * upper_leg_len * math.sin(walk_angle)),
                int(hip[1] + upper_leg_len * math.cos(walk_angle)),
            )
            ankle = (
                int(knee[0] + side * lower_leg_len * math.sin(walk_angle + 0.5)),
                int(knee[1] + lower_leg_len * math.cos(walk_angle + 0.5)),
            )

            cv2.line(frame, hip, knee, (0, 0, 0), 2)
            cv2.line(frame, knee, ankle, (0, 0, 0), 2)

    def update(self):
        self.angle += 0.2
        self.walk_offset += 0.2
        self.color_cnt += 1
        if self.color_cnt % 10 == 0:
            self.color_val = (
                np.random.randint(0, 255),
                np.random.randint(0, 255),
                np.random.randint(0, 255),
            )


def main1(action="walk_and_wave"):
    figure = StickFigure(size=150, position=(250, 300))

    while True:
        frame = np.ones((600, 500, 3), dtype=np.uint8) * 255

        figure.draw(
            frame,
            wave=(action in ["wave", "walk_and_wave"]),
            walk=(action in ["walk", "walk_and_wave"]),
        )
        figure.update()

        cv2.imshow("Stick Figure", frame)
        if cv2.waitKey(50) & 0xFF == ord("q"):
            break

    cv2.destroyAllWindows()


def main2():
    import os

    out_folder = "out_images"
    os.makedirs(out_folder, exist_ok=True)
    cnt = 0
    frames_list = []
    w = 600
    h = 500
    figure = StickFigure(size=150, position=(250, 300))
    smile_image = cv2.imread("data/smile.jpg")
    smile_image = cv2.resize(smile_image, (w, h))
    for action in [
        "walk",
        "wave",
        "walk_and_wave",
        "walk_and_wave",
        "walk_and_wave",
        "SMILE",
    ]:
        for _ in range(50):
            frame = np.ones((w, h, 3), dtype=np.uint8) * 255

            if action == "SMILE":
                frame = smile_image
            # while True:
            else:
                figure.draw(
                    frame,
                    wave=(action in ["wave", "walk_and_wave"]),
                    walk=(action in ["walk", "walk_and_wave"]),
                )
                figure.update()

            cv2.imwrite(f"{out_folder}/{cnt:04}.png", frame)
            cnt += 1
            cv2.imshow("Stick Figure", frame)
            if cv2.waitKey(50) & 0xFF == ord("q"):
                break

    cv2.destroyAllWindows()


if __name__ == "__main__":
    main2()

    # for action_mode in ["walk", "wave","walk_and_wave" ]:
    # # for action_mode in ["walk_and_wave"]:
    #     # action_mode = 'walk_and_wave'  # ← change this for different animations
    #     main1(action=action_mode)

# images to video

In [15]:
# Specify the folder containing images and the output video file name
import os


image_folder = 'out_images'
video_name = 'out_video.mp4'

# Get the list of images in the folder
images = [img for img in os.listdir(image_folder) if img.endswith(".png") or img.endswith(".jpg")]
# images.sort(reverse=True)  # Sort images if needed

# Check if there are images
if not images:
    print("No images found in the specified folder.")
    exit()

# Read the first image to get the width and height
first_image = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = first_image.shape

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can use 'MJPG' or 'XVID'
video = cv2.VideoWriter(video_name, fourcc, 30, (width, height))  # 30 is the frame rate

# Loop through the images and write them to the video
for image in images:
    print(image)
    img_path = os.path.join(image_folder, image)
    frame = cv2.imread(img_path)
    if frame is None:
        print(f"Warning: {image} could not be read.")
        continue  # Skip this image if it cannot be read
    
    frame = cv2.resize(frame, (width, height))
    video.write(frame)  # Write the frame to the video
    print(f"Added {image} to video.")  # Debugging output

# Release the video writer object
video.release()
cv2.destroyAllWindows()

0000.png
Added 0000.png to video.
0001.png
Added 0001.png to video.
0002.png
Added 0002.png to video.
0003.png
Added 0003.png to video.
0004.png
Added 0004.png to video.
0005.png
Added 0005.png to video.
0006.png
Added 0006.png to video.
0007.png
Added 0007.png to video.
0008.png
Added 0008.png to video.
0009.png
Added 0009.png to video.
0010.png
Added 0010.png to video.
0011.png
Added 0011.png to video.
0012.png
Added 0012.png to video.
0013.png
Added 0013.png to video.
0014.png
Added 0014.png to video.
0015.png
Added 0015.png to video.
0016.png
Added 0016.png to video.
0017.png
Added 0017.png to video.
0018.png
Added 0018.png to video.
0019.png
Added 0019.png to video.
0020.png
Added 0020.png to video.
0021.png
Added 0021.png to video.
0022.png
Added 0022.png to video.
0023.png
Added 0023.png to video.
0024.png
Added 0024.png to video.
0025.png
Added 0025.png to video.
0026.png
Added 0026.png to video.
0027.png
Added 0027.png to video.
0028.png
Added 0028.png to video.
0029.png
Added

In [16]:
cap = cv2.VideoCapture("out/%04d.png")
while cap.isOpened():
    ret,frame=cap.read()
    if not ret:
        break